In [9]:

# boolean network packages
from sad2_final_project.boolean_bn import BN
from sad2_final_project.boolean_bn import save_trajectories_to_bnfinder_format 
from boolean import BooleanAlgebra 
# system packages
import os
from pathlib import Path


# paths
## set global dir
cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())
## create directories
!mkdir -p 'data/test/datasets'
!mkdir -p 'data/test/bn_ground_truth'
!mkdir -p 'data/test/bn_predicted'
## create paths
DATA_PATH = Path('data/test')
DATASET_PATH = DATA_PATH / 'datasets'
BN_TRUE_PATH = DATA_PATH / 'bn_ground_truth'
BN_PREDICTED_PATH = DATA_PATH / 'bn_predicted'




/home/maxi7524/repositories/SAD2_final_project


## Part 1 

### 1. Construct boolean networks (way to do it ?)
#### Task
Construct several Boolean networks with sizes (measured by the number of nodes or variables) ranging from 5 to 16.† Each node should have no more than three parent nodes, and the Boolean functions governing individual nodes should be generated at random.

#### Ideas

In [ ]:
# TODO konstrukcja sieci

## TODO - tworzymy generator który będzie nam zwracał odpowiednie sieci

## TODO - losujemy na zadanych warunkach nasze sieci

## zapisujemy nasze wygenerowane funkcje w klasie bn (można domyślnie) 


# BRUDNO 
algebra = BooleanAlgebra()
x1 = algebra.Symbol('x0')
x2 = algebra.Symbol('x1')
x3 = algebra.Symbol('x2')

# Define functions (the same as in exercise in lab 4)
f1 = x2
f2 = ~x2
f3 = ~x2 | x3

functions = [f1, f2, f3]

# Create BN with fixed functions - od razu mamy całość 
bn = BN(num_nodes=3, mode="asynchronous", trajectory_length=50, functions=functions)


2026-01-19 02:41:06,701 [INFO] sad2_final_project.boolean_bn.bn: Initializing Boolean Network with 3 nodes in asynchronous mode.
2026-01-19 02:41:06,703 [INFO] sad2_final_project.boolean_bn.bn: Boolean Network initialized successfully.
2026-01-19 02:41:06,704 [INFO] sad2_final_project.boolean_bn.bn: Computing attractors.
2026-01-19 02:41:06,705 [INFO] sad2_final_project.boolean_bn.bn: Generating state transition system (STS).
2026-01-19 02:41:06,706 [INFO] sad2_final_project.boolean_bn.bn: State transition system generated with 8 states and 21 transitions.
2026-01-19 02:41:06,707 [INFO] sad2_final_project.boolean_bn.bn: Found 1 attractors.


[{(1, 0, 1), (0, 0, 1), (1, 1, 1), (0, 1, 1)}]


### 2. Simulate boolean networks 
#### Task
Simulate trajectories of the generated networks in both synchronous and asynchronous modes to create datasets. The datasets should vary in 
- the proportion of transient and attractor states,
- the trajectory sampling frequency (i.e. the number of time steps between consecutive sampled states), and 
- their overall size (i.e. the number and length of trajectories used to construct each dataset);


In [10]:
# TODO symulacja sieci

##  Korzystamy z funkcji Michała, do symulacji
save_trajectories_to_bnfinder_format(bn_instance=bn, num_trajectories=60, output_file= DATASET_PATH / 'condition_1' )

## TODO musimy zrobić dla każdej sieci różne datasety, to jet network_1_condition_x i mamy x warunków dla każdego przypadku, żeby zobaczyć jak się zachowuje znajdywanie takiej sieci w zależności od dataset'u 

2026-01-19 02:50:08,327 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-19 02:50:08,331 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (1/100)
2026-01-19 02:50:08,333 [WARNING] sad2_final_project.boolean_bn.bn: Max attempts exceeded at step 1, restarting trajectory (3/100)
2026-01-19 02:50:08,337 [INFO] sad2_final_project.boolean_bn.bn: Trajectory completed. Length: 50, Attractors: 57, Transients: 90
2026-01-19 02:50:08,343 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-19 02:50:08,346 [INFO] sad2_final_project.boolean_bn.bn: Trajectory completed. Length: 50, Attractors: 64, Transients: 83
2026-01-19 02:50:08,347 [INFO] sad2_final_project.boolean_bn.bn: Simulating trajectory with target attractor ratio 0.40, tolerance 0.10, max_iter 50
2026-01-19 02:50:08,353 [INFO] 

Done.


### 3. Build dynamic Bayesian newtorks BNFinder2
#### Task
Use the datasets to infer dynamic Bayesian networks with the BNFinder2 software tool. Unlike classical Bayesian networks, dynamic Bayesian networks can contain cycles, making them more suitable for this task. Consider two scoring functions (referred to as scoring criteria in the BNFinder2 terminology): Minimal Description Length (MDL) and BDe (Bayesian–Dirichlet equivalence).

In [ ]:
# TODO odbudowa sieci 

## TODO bierezmy data sety z poprzedniego ćwiczenia i je tutaj odtwarzamy za pomocą tego skryptu, więc powinniśmy móc go po prostu uruchomić
from sad2_final_project.bnfinder import run_bnfinder

datasets = [DATASET_PATH / 'condition_1',]
for d in datasets:
    run_bnfinder(
        data=d,
        ground_truth="gt.csv",
        output_template="results/{}_{}.sif"
    )
## TODO przeliczenie miar (to trzeba osobny skrypt na to, bo ja bym to chciał zestawić potem w jedną tabele)

### 4. Asses Quaility 
#### Task
Evaluate the accuracy of the reconstructed network structures with respect to the characteristics of the datasets and the scoring functions used. Consider the original Boolean network (i.e. the one used to generate the dataset) as the ground truth. The evaluation should employ at least two structure-based graph distance measures of your choice, with a brief justification for the selected measures.

In [ ]:
# TODO - tutaj tylko wytłumaczeni wniosków bo poprzednoi to jest metodologia

## Part 2

### 1. Choose validated Boolean network
#### Task
our task is to consider a validated Boolean network model of a real-life biological mechanism. To this end, select a Boolean network 2 model of your choice from the ‘models’ subfolder of the Biodivine repository, available at https://github.com/sybila/biodivine-boolean-models, with the number of nodes (variables) not exceeding 16.‡. 

In [ ]:
# TODO - wybrac sieć

# TODO - 

### 2. Generate dataset,
#### Task
Using the insights gained from the first part of the project,
generate an appropriate dataset for the network inference task. 




In [ ]:
# TODO - tworzymy dataset (na bazie wniosków z poprzedniego)
# TODO - tutaj bierzemy wszystkie datasety, jakie wnioski udało nam się ustalić do nich

### 3. Reconstruct the network with BNFinder2 
#### Task
Reconstruct the network structure with BNFinder2, applying a scoring function chosen based on your previous
experience. Evaluate the accuracy of the reconstruction.

In [ ]:
# TODO zliczenie jakości dopasowania

# Notes

Idea behind structure is:

# Report

## Part I
## Part II

# Bibliografy